# Introduction to UNIQ for DNAC
## Overview

This notebook shows how to use the uniq python library with DNAC.



## Getting Started
The first requirement is to import a NbClientManager to connect to DNAC

In [ ]:
import json
import requests.exceptions

# this is the client manager used to connect to DNAC
from uniq.apis.nb.client_manager import NbClientManager

# this will be the ip/dns name of DNAC, the user/password credentials
# stored in current directory
from dnac_config import DNAC, DNAC_USER, DNAC_PASSWORD

Next we create a connection, and check for obvious errors.


In [ ]:
# create the connection to DNAC
try:
    dnac = NbClientManager(
                server=DNAC,
                username=DNAC_USER,
                password=DNAC_PASSWORD,
                connect=True)

except requests.exceptions.HTTPError as exc_info:
    if exc_info.response.status_code == 401:
        print('Authentication Failed. Please provide valid username/password.')
    else:
        print('HTTP Status Code {code}. Reason: {reason}'.format(
                    code=exc_info.response.status_code,
                    reason=exc_info.response.reason))
    exit(1)
except requests.exceptions.ConnectionError:
    print('Connection aborted. Please check if the host {host} is available.'.format(host=DNAC))
    exit(1)

If you print the connection, you will see the server/username/password and version of the API

In [ ]:
print(dnac)

## First API Request
This example gets all of the network devices from the controller inventory.  

### Accessing network-devices on DNAC

In [ ]:
# this method will get all network devices
network_devices  = dnac.networkdevice.getAllNetworkDevice()

In [ ]:
# now pretty print it
# serialize turns object into a json dict.
# we do not normally do this, but shows the structure maps to a dictionary
print(json.dumps(dnac.serialize(network_devices), indent=2))

### Displaying data

Now print out a subset of the attributes.  Define a format string that can be used for both the heading and the data.

- UseCase: Print out a comma sepparated list of attributes for importing into an asset management system 

In [ ]:
# define a formating string
formatstring='{ip:<16s}, {name:<30s}, {software:<16s}, {serial:<20s}'


# print a heading
print(formatstring.format(ip="IP Address", 
                          name="Device Name", 
                          software="Software Version",
                         serial="Serial Number"))

# print each of the nework devices.  network_devices is a list of objects with attributes, not a python dict
for network_device in network_devices.response:
    print(formatstring.format(ip=network_device.managementIpAddress, 
                                 name=network_device.hostname,
                                 software=network_device.softwareVersion,
                             serial=network_device.serialNumber))

<font color=blue>
<hr>
<h2> Challenge </h2>
Either insert a cell or modify the cell above to change the attributes that are printed out.
<hr>
</font>

## Getting a Specific resource

Now find a specific device and make a request for a single device


In [ ]:
[(net_dev.id, net_dev.upTime) for net_dev in network_devices.response ]

In [ ]:
# look for a specific access device which is a 9300
try:
    # one line list comprehension
    deviceId = [net_dev.id for net_dev in network_devices.response 
                 if net_dev.role =="ACCESS" and  "9300" in net_dev.type ][0]
    print ("Success, found id:{id}".format(id=deviceId))
except keyError:
    print ("FAIL: no suitable device found")

This id can be used to return all the information about a single device.  Device id are important as the uniquely identify the device.  They are used in many API calls to perform an action on a device.  For example to assign a tag or a location for a device, you would need to know the ```id```.

In [ ]:
network_device = dnac.networkdevice.getNetworkDeviceById(id=deviceId)
print (json.dumps(dnac.serialize(network_device), indent=2))

### Get Network-device by management IP address
Looking a device up by id is challenging as you may not always know the id.  It is often the case you know the management IP address (or serialnumber) and want to get the id of the device, or some other attributes (e.g. version of code).

- UseCase: a function to allow the user to enter the IP address of a network-device and perform some other action based on the id.

In [ ]:
# looks up a network device by ipaddress, and returns the id attribute
def ipToDeviceId(ipAddress):
    ip_network_device = dnac.networkdevice.getNetworkDeviceByIp(ipAddress=ipAddress)
    return ip_network_device.response.id

# we are cheating here as we are just picking the IP address from earlier
# this is just to illustrate the use of a known ip address
ipAddress=network_device.response.managementIpAddress
id1 = ipToDeviceId(ipAddress)
print ("Example1: %s" % id1)

# now use a specific IP address.  You can change this to one of the examples above
ipAddress="10.10.22.70"
id2 = ipToDeviceId(ipAddress)
print ("Example1: %s" % id2)

<hr>
<font color=blue>
<h2> Challenge </h2>
Write some code to get all network-devices running IOS "15.2(4)M6a"
</font>
<hr>

## Host data store
This call show the hosts connected to the network.

- UseCase: find my host


In [ ]:
hosts = dnac.host.getHosts()
print(json.dumps(dnac.serialize(hosts), indent=4))

Lookup a device by ipaddress (macAddress, hostname).  
Can also use connectedInterfaceName and connectedNetworkDeviceIpAddress

In [ ]:
# lookup host by IP address
host=dnac.host.getHosts(hostIp="10.10.22.114").response[0]

print("host: {ip} {mac}: DEVICE{deviceIp} {interface}".format(
                        ip=host.hostIp,
                        mac=host.hostMac,
                        deviceIp=host.connectedNetworkDeviceIpAddress,
                        interface=host.connectedInterfaceName   ))

<hr>
<font color=blue>
<h2> Challenge </h2>
Write some code to get the IOS version of the device a host is connected to.
</font>
<hr>

### interfaces

Look at the interfaces that the device above is connected to.  Get switch port utilization


In [ ]:
host=dnac.host.getHosts(hostIp="10.10.22.114").response[0]
deviceId =host.connectedNetworkDeviceId
interfaces = dnac.interface.getInterfaceByDeviceId(deviceId=deviceId).response

In [ ]:
print (json.dumps(dnac.serialize(interfaces), indent=4))

In [ ]:
# get a list of portName,  speed and status
[(port.portName, port.speed, port.status)
                 for port in interfaces
                 if port.interfaceType == 'Physical']

In [ ]:
import re
def atoi(text):
    return int(text) if text.isdigit() else text

# natural sort for interfaces
def natural_sort(interfacelist):
    return sorted(interfacelist, key=lambda port: [ atoi(c) for c in re.split('(\d+)', port.portName)])

[(port.portName, port.speed, port.status)
        for port in natural_sort(interfaces)
            if port.interfaceType == 'Physical']

In [ ]:
len([port.portName for port in natural_sort(interfaces)
                 if port.interfaceType == 'Physical'
                    and port.status == 'up'])

<hr>
<font color=blue>
<h2> Challenge </h2>
Calculate the percentage utilization (active ports) for the switch above.
</font>
<hr>